In [ ]:
from google.colab import drive
import os

# 强制重新挂载，防止玄学问题
drive.mount('/content/drive', force_remount=True)

# 定义你的项目根目录（请确保 Drive 里有 car_project 文件夹）
project_root = '/content/drive/MyDrive/car_project'
dataset_root = os.path.join(project_root, 'dataset')

print(f"项目路径已设定为: {project_root}")

Mounted at /content/drive
项目路径已设定为: /content/drive/MyDrive/car_project


In [2]:
#缺失补充
# 1. 强制单独安装 loguru (这是你报错缺少的那个)
!pip install loguru

# 2. 安装其他必不可少的库 (thop用于计算算力，tabulate用于打印表格)
!pip install thop
!pip install tabulate
!pip install ninja

# 3. 再次验证是否安装成功
try:
    import loguru
    import thop
    print("---------------------------------------------")
    print(f"✅ 成功！Loguru 版本: {loguru.__version__}")
    print("✅ 成功！环境修复完成，可以继续后面的步骤了！")
    print("---------------------------------------------")
except ImportError as e:
    print(f"❌ 依然报错: {e}")
    print("如果你看到这一行，请尝试点击菜单栏的【代码执行程序】->【重新启动会话】，然后重新运行这一格。")

  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 682.8 kB/s eta 0:00:00
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 1.1 MB/s eta 0:00:00
---------------------------------------------
✅ 成功！Loguru 版本: 0.7.3
✅ 成功！环境修复完成，可以继续后面的步骤了！
---------------------------------------------


In [3]:
# 1. 挂载云盘 (因为你重启了)整合
#from google.colab import drive
import os
#drive.mount('/content/drive')

# 2. 补齐缺失的库 (解决 pip 报错的问题)
!pip install loguru thop tabulate ninja onnx onnx-simplifier

# 3. 重新下载并安装 YOLOX (因为切换机器后 YOLOX 文件夹没了)
%cd /content
if not os.path.exists("YOLOX"):
    !git clone https://github.com/Megvii-BaseDetection/YOLOX.git

%cd /content/YOLOX
# 安装 YOLOX 本体
!pip install -e . --no-deps

# 4. 验证 GPU 是否开启
import torch
if torch.cuda.is_available():
    print("------------------------------------------------")
    print(f"✅ GPU 准备就绪！设备名: {torch.cuda.get_device_name(0)}")
    print("------------------------------------------------")
else:
    print("❌ 依然是 CPU 模式，请去【代码执行程序】->【更改运行时类型】里选 GPU！")

  Using cached onnx-1.20.0-cp312-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
  Using cached onnx-simplifier-0.4.36.tar.gz (21.0 MB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/6a/95/9d93b8cfdd9f57abe7000cd6b9e56e2c518ce0e6bf6b312b1cf37b4e68a8/onnx-simplifier-0.4.36.tar.gz (from https://pypi.org/simple/onnx-simplifier/) (requires-python:>=3.7): Requested onnxsim from https://files.pythonhosted.org/packages/6a/95/9d93b8cfdd9f57abe7000cd6b9e56e2c518ce0e6bf6b312b1cf37b4e68a8/onnx-simplifier-0.4.36.tar.gz has inconsistent name: expected 'onnx-simplifier', but metadata has 'onnxsim'
  Using cached onnx-simplifier-0.4.35.tar.gz (20.1 MB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/a5/65/56398c020ea361a879968d8f703ccf34fcb4ad172eff17b132f6948271ad/onnx-simplifier-0.4.35.tar.gz (from https://pypi.org/simple/onnx-simplifier/) (requires-python:>=3.7): Requested onnxsim from h

In [ ]:
import os
import cv2
#生成xml，已完成
# 输入：你的原始图片文件夹 (请确认路径正确！)
raw_img_dir = os.path.join(dataset_root, 'raw')
# 输出：XML 保存位置
save_xml_dir = os.path.join(dataset_root, 'xmls')

if not os.path.exists(save_xml_dir): os.makedirs(save_xml_dir)

def generate_xml_content(img_name, w, h, xmin, ymin, xmax, ymax):
    return f"""<annotation>
    <folder>CCPD</folder>
    <filename>{img_name}</filename>
    <size>
        <width>{w}</width>
        <height>{h}</height>
        <depth>3</depth>
    </size>
    <object>
        <name>license_plate</name>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{ymin}</ymin>
            <xmax>{xmax}</xmax>
            <ymax>{ymax}</ymax>
        </bndbox>
    </object>
</annotation>"""

print("开始生成 XML...")
count = 0
for filename in os.listdir(raw_img_dir):
    if not filename.endswith('.jpg'): continue
    try:
        # 解析坐标
        parts = filename.split('-')
        bbox = parts[2].split('_')
        x1, y1 = map(int, bbox[0].split('&'))
        x2, y2 = map(int, bbox[1].split('&'))

        # 读取图片宽高
        img = cv2.imread(os.path.join(raw_img_dir, filename))
        if img is None: continue
        h, w = img.shape[:2]

        # 写入 XML
        xml_content = generate_xml_content(filename, w, h, min(x1,x2), min(y1,y2), max(x1,x2), max(y1,y2))
        xml_name = filename.replace('.jpg', '.xml')
        with open(os.path.join(save_xml_dir, xml_name), 'w') as f:
            f.write(xml_content)
        count += 1
    except: pass

print(f"✅ 处理完成，共生成 {count} 个 XML 文件。")

In [ ]:
import shutil
#生成train.txt 已完成
# 定义 VOC 结构路径
voc_root = os.path.join(dataset_root, 'VOC2007')
anno_dir = os.path.join(voc_root, 'Annotations')
jpeg_dir = os.path.join(voc_root, 'JPEGImages')
txt_dir = os.path.join(voc_root, 'ImageSets/Main')

os.makedirs(anno_dir, exist_ok=True)
os.makedirs(jpeg_dir, exist_ok=True)
os.makedirs(txt_dir, exist_ok=True)

print("正在整理文件夹结构（复制文件需要一点时间）...")

# 1. 复制 XML
xml_src = os.path.join(dataset_root, 'xmls')
for f in os.listdir(xml_src):
    shutil.copy(os.path.join(xml_src, f), os.path.join(anno_dir, f))

# 2. 复制图片
img_src = os.path.join(dataset_root, 'raw')
for f in os.listdir(img_src):
    if f.endswith('.jpg'):
        shutil.copy(os.path.join(img_src, f), os.path.join(jpeg_dir, f))

# 3. 生成 train.txt
xml_files = [f[:-4] for f in os.listdir(anno_dir) if f.endswith('.xml')]
with open(os.path.join(txt_dir, 'train.txt'), 'w') as f:
    f.write('\n'.join(xml_files))

print(f"✅ 数据准备完毕！训练集包含 {len(xml_files)} 张图片。")
print(f"train.txt 位置: {os.path.join(txt_dir, 'train.txt')}")

In [ ]:
#生成配置文件（上传）/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py
config_content = f"""
import os
from yolox.exp import Exp as MyExp

class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]

        # --- 关键修改 ---
        self.num_classes = 1
        self.max_epoch = 50
        self.data_dir = "{dataset_root}"  # 自动填入你的路径
        self.train_ann = "train.txt"
        self.val_ann = "train.txt"        # 为了演示，验证集也用这个
        # ----------------

    def get_dataset(self, cache: bool, cache_type: str = "ram"):
        from yolox.data import VOCDetection, TrainTransform
        return VOCDetection(
            data_dir=self.data_dir,
            image_sets=[('2007', 'train')], # 读取 VOC2007/ImageSets/Main/train.txt
            img_size=self.input_size,
            preproc=TrainTransform(
                max_labels=50,
                flip_prob=0.5,
                hsv_prob=1.0),
            cache=cache,
            cache_type=cache_type,
        )
"""

# 将配置写入文件
with open('/content/YOLOX/exps/example/yolox_voc/yolox_plate.py', 'w') as f:
    f.write(config_content)

print("✅ 配置文件 yolox_plate.py 已生成！")

In [ ]:
#重新生成xml，处理转义字符，已处理
import os
import cv2

# ================= 路径配置 =================
# 你的 VOC2007 根目录
voc_root = '/content/drive/MyDrive/car_project/dataset/VOC2007'

# 图片在哪里？
img_dir = os.path.join(voc_root, 'JPEGImages')
# XML 要存哪里？(原地覆盖)
xml_dir = os.path.join(voc_root, 'Annotations')

# 确保目标目录存在
os.makedirs(xml_dir, exist_ok=True)
# ===========================================

def generate_xml_content(img_name, w, h, xmin, ymin, xmax, ymax):
    # 【核心修复】将文件名中的 & 替换为 &amp;
    # 这样 XML 解析器就不会报错了
    safe_name = img_name.replace("&", "&amp;")

    return f"""<annotation>
    <folder>CCPD</folder>
    <filename>{safe_name}</filename>
    <path>{safe_name}</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>{w}</width>
        <height>{h}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>license_plate</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{ymin}</ymin>
            <xmax>{xmax}</xmax>
            <ymax>{ymax}</ymax>
        </bndbox>
    </object>
</annotation>"""

print(f"正在读取图片目录: {img_dir}")
print(f"准备修复 XML 到: {xml_dir}")

count = 0
error_count = 0

# 直接遍历 JPEGImages 里的文件
for filename in os.listdir(img_dir):
    if not filename.endswith('.jpg'):
        continue

    try:
        # 1. 解析文件名 (CCPD 格式)
        parts = filename.split('-')
        # 防止部分文件名格式不对导致报错，加个判断
        if len(parts) < 3:
            continue

        bbox = parts[2].split('_')
        x1, y1 = map(int, bbox[0].split('&'))
        x2, y2 = map(int, bbox[1].split('&'))

        # 2. 读取图片获取宽高
        # (虽然慢一点，但为了保证宽高数据准确，建议读取)
        img_path = os.path.join(img_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        # 3. 生成 XML 内容
        xml_content = generate_xml_content(filename, w, h, min(x1,x2), min(y1,y2), max(x1,x2), max(y1,y2))

        # 4. 写入 XML 文件 (文件名保持一致，后缀变 .xml)
        xml_name = filename.replace('.jpg', '.xml')
        save_path = os.path.join(xml_dir, xml_name)

        with open(save_path, 'w') as f:
            f.write(xml_content)

        count += 1
        if count % 200 == 0:
            print(f"已修复 {count} 个文件...")

    except Exception as e:
        # 打印一下具体是哪个文件出错，方便排查，但不中断程序
        # print(f"跳过坏文件 {filename}: {e}")
        error_count += 1

print("------------------------------------------------")
print(f"✅ 修复完成！")
print(f"成功覆盖/生成 XML: {count} 个")
print(f"失败/跳过: {error_count} 个")
print("------------------------------------------------")
print("现在可以直接去运行训练命令了！")

正在读取图片目录: /content/drive/MyDrive/car_project/dataset/VOC2007/JPEGImages
准备修复 XML 到: /content/drive/MyDrive/car_project/dataset/VOC2007/Annotations
已修复 200 个文件...
已修复 400 个文件...
已修复 600 个文件...
已修复 800 个文件...
已修复 1000 个文件...
已修复 1200 个文件...
已修复 1400 个文件...
已修复 1600 个文件...
已修复 1800 个文件...
已修复 2000 个文件...
已修复 2200 个文件...
已修复 2400 个文件...
已修复 2600 个文件...
已修复 2800 个文件...
已修复 3000 个文件...
已修复 3200 个文件...
已修复 3400 个文件...
已修复 3600 个文件...
已修复 3800 个文件...
已修复 4000 个文件...
已修复 4200 个文件...
已修复 4400 个文件...
已修复 4600 个文件...
已修复 4800 个文件...
已修复 5000 个文件...
已修复 5200 个文件...
已修复 5400 个文件...
已修复 5600 个文件...
------------------------------------------------
✅ 修复完成！
成功覆盖/生成 XML: 5780 个
失败/跳过: 0 个
------------------------------------------------
现在可以直接去运行训练命令了！


In [ ]:
#识别license-plate
import os
import glob

# 定义我们要修改的目录
target_dir = '/content/YOLOX/yolox/data/datasets'

# 我们要替换成这个（注意格式，只有一个元素时后面要加逗号）
new_classes_def = 'VOC_CLASSES = ( "license_plate", )'

# 1. 自动搜索包含 "aeroplane" 的文件
print(f"正在 {target_dir} 里寻找定义类别的文件...")
target_file = None

for filepath in glob.glob(os.path.join(target_dir, "*.py")):
    with open(filepath, 'r') as f:
        content = f.read()
        if '"aeroplane"' in content or "'aeroplane'" in content:
            target_file = filepath
            break

if target_file:
    print(f"✅ 找到了！类别定义在: {target_file}")

    # 2. 读取并暴力替换
    with open(target_file, 'r') as f:
        lines = f.readlines()

    new_content = []
    is_skipping = False

    for line in lines:
        stripped = line.strip()

        # 开始替换逻辑
        if stripped.startswith("VOC_CLASSES = ("):
            new_content.append(new_classes_def + "\n")
            is_skipping = True # 开始跳过原来的很多行

        if is_skipping:
            # 如果遇到了右括号 )，说明旧的列表结束了，停止跳过
            if stripped.startswith(")"):
                is_skipping = False
            continue # 跳过旧的行

        new_content.append(line)

    # 3. 写回文件
    with open(target_file, 'w') as f:
        f.writelines(new_content)

    print("✅ 修改完成！")

    # 4. 验证一下
    print("---------------- 文件内容预览 ----------------")
    os.system(f"grep 'VOC_CLASSES' {target_file}")
    print("---------------------------------------------")

else:
    print("❌ 奇怪，找不到定义 aeroplane 的文件。你确定 YOLOX 安装完整了吗？")
    # 如果找不到，强制在 voc.py 里追加一行（兜底方案）
    fallback_file = os.path.join(target_dir, 'voc.py')
    print(f"⚠️ 尝试强制追加到 {fallback_file}")
    with open(fallback_file, 'a') as f:
        f.write("\n" + new_classes_def + "\n")

正在 /content/YOLOX/yolox/data/datasets 里寻找定义类别的文件...
✅ 找到了！类别定义在: /content/YOLOX/yolox/data/datasets/voc_classes.py
✅ 修改完成！
---------------- 文件内容预览 ----------------
---------------------------------------------


In [ ]:
#numpy报错
import os

# 目标文件
target_file = '/content/YOLOX/yolox/data/datasets/voc.py'

print(f"正在修复文件: {target_file}")

with open(target_file, 'r') as f:
    content = f.read()

# 【核心修复】
# 把 "if dets == []:" 替换为 "if len(dets) == 0:"
old_code = "if dets == []:"
new_code = "if len(dets) == 0:"

if old_code in content:
    new_content = content.replace(old_code, new_code)

    with open(target_file, 'w') as f:
        f.write(new_content)
    print("✅ 修复成功！NumPy 不会再报错了。")
else:
    print("⚠️ 未找到目标代码，可能已经修复过了，或者代码版本不同。")
    # 尝试打印一下相关上下文看看
    start_idx = content.find("_write_voc_results_file")
    if start_idx != -1:
        print("相关代码片段预览：")
        print(content[start_idx:start_idx+500])

正在修复文件: /content/YOLOX/yolox/data/datasets/voc.py
✅ 修复成功！NumPy 不会再报错了。


In [ ]:
#置信度读取
import os

# 目标文件
target_file = '/content/YOLOX/yolox/evaluators/voc_eval.py'

print(f"正在修复验证脚本: {target_file}")

with open(target_file, 'r') as f:
    content = f.read()

# 核心修复逻辑：
# 将 confidence = np.array([float(x[1]) for x in splitlines])
# 修改为 confidence = np.array([float(x[-5]) for x in splitlines])

old_code = "float(x[1])"
new_code = "float(x[-5])" # 取倒数第5个，永远是置信度

if old_code in content:
    new_content = content.replace(old_code, new_code)

    with open(target_file, 'w') as f:
        f.write(new_content)
    print("✅ 修复成功！现在它可以处理带空格的文件名了 (如 'image (1).jpg')。")
else:
    print("⚠️ 未找到目标代码，可能已经修复过了。")
    # 检查是否已经是 -5
    if "float(x[-5])" in content:
        print("确认代码已经是修复状态 (x[-5])。")

正在修复验证脚本: /content/YOLOX/yolox/evaluators/voc_eval.py
✅ 修复成功！现在它可以处理带空格的文件名了 (如 'image (1).jpg')。


In [ ]:
#位置报错
import os

# 目标文件
target_file = '/content/YOLOX/yolox/evaluators/voc_eval.py'

print(f"正在修复坐标提取逻辑: {target_file}")

with open(target_file, 'r') as f:
    content = f.read()

# 核心修复逻辑：
# 将 [float(z) for z in x[2:]]
# 修改为 [float(z) for z in x[-4:]]

old_code = "for z in x[2:]]"
new_code = "for z in x[-4:]]"

if old_code in content:
    new_content = content.replace(old_code, new_code)

    with open(target_file, 'w') as f:
        f.write(new_content)
    print("✅ 修复成功！坐标提取逻辑已改为取最后4位 (x[-4:])。")
    print("这下彻底兼容所有带空格、括号的文件名了。")
else:
    print("⚠️ 未找到目标代码，可能已经修复过了。")
    # 检查是否已经是 -4
    if "x[-4:]" in content:
        print("确认代码已经是修复状态 (x[-4:])。")

正在修复坐标提取逻辑: /content/YOLOX/yolox/evaluators/voc_eval.py
✅ 修复成功！坐标提取逻辑已改为取最后4位 (x[-4:])。
这下彻底兼容所有带空格、括号的文件名了。


In [ ]:
#数据集到临时目录
import shutil
import os
import time

# 1. 源路径 (在 Drive 上，读取慢)
drive_dataset_dir = "/content/drive/MyDrive/car_project/dataset"

# 2. 目标路径 (在 Colab 本地，读取快)
local_dataset_dir = "/content/dataset_local"

print(f"正在从 Google Drive 复制数据到本地: {local_dataset_dir} ...")
print("这可能需要几分钟，但磨刀不误砍柴工！")

start_time = time.time()

# 如果本地已经有了，先删掉防止混淆
if os.path.exists(local_dataset_dir):
    shutil.rmtree(local_dataset_dir)

# 开始复制
shutil.copytree(drive_dataset_dir, local_dataset_dir)

end_time = time.time()
print(f"✅ 复制完成！耗时: {end_time - start_time:.2f} 秒")
print("现在的读取速度将飞起！")

正在从 Google Drive 复制数据到本地: /content/dataset_local ...
这可能需要几分钟，但磨刀不误砍柴工！
✅ 复制完成！耗时: 273.35 秒
现在的读取速度将飞起！


In [ ]:
import os

os.makedirs("/content/YOLOX/YOLOX_outputs/yolox_voc_s",exist_ok=True)

In [ ]:
#重命名问题
import os

# 这是目前 YOLOX 正在读取的本地路径
target_dir = '/content/dataset_local/VOC2007/JPEGImages'

print(f"正在对 {target_dir} 进行地毯式搜索...")
print("搜索条件：文件名中包含 '空格' 或 '括号' ...")

suspicious_files = []

if os.path.exists(target_dir):
    for filename in os.listdir(target_dir):
        # 检查文件名里是否有 空格、(、)
        if ' ' in filename or '(' in filename or ')' in filename:
            suspicious_files.append(filename)
else:
    print("❌ 找不到目录，请确认你是否已经把数据复制到本地了。")

print("------------------------------------------------")
if len(suspicious_files) == 0:
    print("✅ 居然没有？那太奇怪了。")
    print("如果文件名都很干净，那可能是生成的 txt 结果文件格式坏了。")
else:
    print(f"🚨 破案了！找到了 {len(suspicious_files)} 个不规范的文件：")
    print("前 10 个嫌疑犯：")
    for f in suspicious_files[:10]:
        print(f"  -> {f}")

    print("\n【原因分析】")
    print("Python 的 split() 默认按空格切割。")
    print(f"例如文件名叫: {suspicious_files[0]}")
    print(f"切割后变成了: {suspicious_files[0].split()}")
    print("代码本来想取第2个当分数，结果取到了文件名的一部分，所以崩了。")
print("------------------------------------------------")

正在对 /content/dataset_local/VOC2007/JPEGImages 进行地毯式搜索...
搜索条件：文件名中包含 '空格' 或 '括号' ...
------------------------------------------------
🚨 破案了！找到了 11 个不规范的文件：
前 10 个嫌疑犯：
  -> 017669270833333334-90_269-253&512_489&587-489&587_253&587_254&513_488&512-0_0_3_24_32_30_28_30-72-23 (1).jpg
  -> 0177191840278-90_264-256&480_486&558-486&558_262&552_256&483_482&480-0_0_3_24_32_25_33_26-117-119 (1).jpg
  -> 0176649305556-92_264-231&467_451&548-440&548_231&546_241&467_451&477-0_0_3_24_30_32_24_33-95-65 (1).jpg
  -> 0176888020833-90_265-243&380_490&453-490&448_248&453_243&385_489&380-0_0_3_24_33_29_32_29-167-43 (1).jpg
  -> 0176692708333-91_102-203&480_439&555-439&555_216&549_203&480_427&482-0_0_3_24_26_28_27_31-118-77 (1).jpg
  -> 0176974826389-89_270-246&421_479&497-479&495_247&497_246&427_479&421-0_0_3_24_26_29_24_33-133-88 (1).jpg
  -> 0176595052083-91_101-303&465_520&547-520&547_320&541_303&473_508&465-0_0_3_26_27_25_32_25-113-110 (1).jpg
  -> 017669270833333334-90_269-255&474_491&550-491&550_256

In [ ]:
%cd /content/YOLOX
#训练
# 下载预训练权重（加速收敛）
if not os.path.exists("yolox_nano.pth"):
    !wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_nano.pth

# 启动训练
!python tools/train.py \
    -f exps/example/yolox_voc/yolox_voc_s.py \
    -d 1 \
    -b 8 \
    --fp16 \
    -o \
    -c yolox_nano.pth

/content/YOLOX
--2025-12-15 09:07:51--  https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_nano.pth
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/386811486/07b600c6-2a51-41d2-9a32-3424acf6c85d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-15T10%3A07%3A01Z&rscd=attachment%3B+filename%3Dyolox_nano.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-15T09%3A06%3A20Z&ske=2025-12-15T10%3A07%3A01Z&sks=b&skv=2018-11-09&sig=e%2FKYaZjPMTUBLmXtwf976dw8PVxZze%2FdhtArxylJzX0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTc4OTk3MiwibmJmIjoxNzY1Nzg5NjcyLCJwYXRoIjoicmVsZWFz

In [ ]:
#继续训练
!python tools/train.py \
    -f exps/example/yolox_voc/yolox_voc_s.py \
    -d 1 \
    -b 8 \
    --fp16 \
    -o \
    -c /content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth \
    --resume

2025-12-15 09:49:19.098241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765792159.131298   24874 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765792159.140328   24874 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765792159.163662   24874 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765792159.163692   24874 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765792159.163701   24874 computation_placer.cc:177] computation placer alr

In [ ]:

#中途保存
import shutil
import os
import datetime

# 1. 你的 Drive 备份总目录
drive_backup_dir = '/content/drive/MyDrive/car_project/output_backup'

# 2. 你的 YOLOX 训练结果目录 (根据你的截图，就是这个路径)
source_dir = '/content/YOLOX/YOLOX_outputs/yolox_voc_s'

# 3. 生成带时间戳的目标路径 (防止重名报错)
timestamp = datetime.datetime.now().strftime("%m%d_%H%M%S")
target_dir = os.path.join(drive_backup_dir, f"full_backup_{timestamp}")

# 4. 开始备份
if os.path.exists(source_dir):
    print(f"正在把整个 {source_dir} 搬运到 Google Drive...")
    print(f"目标路径: {target_dir}")

    shutil.copytree(source_dir, target_dir)

    print("------------------------------------------------")
    print("✅ 备份成功！")
    print("快去 Google Drive 看看，里面应该有 latest_ckpt.pth 了！")
    print("------------------------------------------------")
else:
    print(f"❌ 找不到源目录: {source_dir}，请确认训练是否启动过。")

正在把整个 /content/YOLOX/YOLOX_outputs/yolox_voc_s 搬运到 Google Drive...
目标路径: /content/drive/MyDrive/car_project/output_backup/full_backup_1215_100325
------------------------------------------------
✅ 备份成功！
快去 Google Drive 看看，里面应该有 latest_ckpt.pth 了！
------------------------------------------------


In [ ]:
import os

# 目标文件
target_file = '/content/YOLOX/tools/export_onnx.py'

print(f"正在修复导出脚本: {target_file}")

with open(target_file, 'r') as f:
    content = f.read()

# 【核心修复】
# 将 torch.onnx._export 替换为 torch.onnx.export
old_code = "torch.onnx._export"
new_code = "torch.onnx.export"

if old_code in content:
    new_content = content.replace(old_code, new_code)

    with open(target_file, 'w') as f:
        f.write(new_content)
    print("✅ 修复成功！已将 '_export' 替换为 'export'。")
else:
    print("⚠️ 未找到目标代码，可能已经修复过了。")

正在修复导出脚本: /content/YOLOX/tools/export_onnx.py
✅ 修复成功！已将 '_export' 替换为 'export'。


In [ ]:
!pip install onnxscript

  Using cached onnx-1.20.0-cp312-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yolox 0.3.0 requires onnx-simplifier==0.4.10, which is not installed.


In [ ]:
#导出
%cd /content/YOLOX

# 1. 找到权重文件
ckpt_file = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth"

# 2. 导出为 ONNX
# 注意：Atlas推理时通常不需要 decode（后处理），但这比较复杂
# 为了最简单跑通，我们先带上 decode 导出
!python tools/export_onnx.py \
    --output-name yolox_plate.onnx \
    -f exps/example/yolox_voc/yolox_voc_s.py \
    -c {ckpt_file}

/content/YOLOX
2025-12-15 10:09:08.128 | INFO     | __main__:main:64 - args value: Namespace(output_name='yolox_plate.onnx', input='images', output='output', opset=11, batch_size=1, dynamic=False, no_onnxsim=False, exp_file='exps/example/yolox_voc/yolox_voc_s.py', experiment_name=None, name=None, ckpt='/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth', opts=[], decode_in_inference=False)
2025-12-15 10:09:08.265 | INFO     | __main__:main:88 - loading checkpoint done.
W1215 10:09:08.668000 29852 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
[torch.onn

In [ ]:
!python tools/export_onnx.py \
    --output-name yolox_plate.onnx \
    -f exps/example/yolox_voc/yolox_voc_s.py \
    -c /content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth \
    --no-onnxsim

2025-12-15 10:10:58.971 | INFO     | __main__:main:64 - args value: Namespace(output_name='yolox_plate.onnx', input='images', output='output', opset=11, batch_size=1, dynamic=False, no_onnxsim=True, exp_file='exps/example/yolox_voc/yolox_voc_s.py', experiment_name=None, name=None, ckpt='/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth', opts=[], decode_in_inference=False)
2025-12-15 10:10:59.104 | INFO     | __main__:main:88 - loading checkpoint done.
W1215 10:10:59.420000 30322 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
[torch.onnx] Obtain model 

In [ ]:
#opset11
%cd /content/YOLOX

# 假设你的权重文件在这里 (请确认路径)
ckpt_file = "/content/YOLOX/latest_ckpt.pth"

# 运行导出命令
# 关键修改：加上了 --opset 11
!python tools/export_onnx.py \
    --output-name yolox_plate_op11.onnx \
    -f exps/example/yolox_voc/yolox_voc_s.py \
    -c {ckpt_file} \
    --opset 11 \
    --no-onnxsim

/content/YOLOX
2025-12-17 13:31:15.472 | INFO     | __main__:main:64 - args value: Namespace(output_name='yolox_plate_op11.onnx', input='images', output='output', opset=11, batch_size=1, dynamic=False, no_onnxsim=True, exp_file='exps/example/yolox_voc/yolox_voc_s.py', experiment_name=None, name=None, ckpt='/content/YOLOX/latest_ckpt.pth', opts=[], decode_in_inference=False)
2025-12-17 13:31:15.616 | INFO     | __main__:main:88 - loading checkpoint done.
W1217 13:31:15.995000 5564 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
[torch.onnx] Obtain model graph f

In [ ]:
import onnx
from onnx.external_data_helper import load_external_data_for_model
import os

# 需要处理的文件列表
files_to_merge = [
    "/content/YOLOX/yolox_plate_op11.onnx",
    "/content/LPRNet_Pytorch/lprnet_standard_op11.onnx"
]

for input_path in files_to_merge:
    if not os.path.exists(input_path):
        print(f"跳过（未找到）：{input_path}")
        continue

    print(f"正在检查合并: {input_path}")
    output_path = input_path.replace(".onnx", "_merged.onnx")

    # 加载
    model = onnx.load(input_path, load_external_data=False)
    load_external_data_for_model(model, base_dir=os.path.dirname(input_path))

    # 保存 (onnx 库会自动处理合并)
    onnx.save(model, output_path)

    print(f"✅ 生成单文件模型: {output_path}")
    print(f"大小: {os.path.getsize(output_path)/1024/1024:.2f} MB\n")

正在检查合并: /content/YOLOX/yolox_plate_op11.onnx
✅ 生成单文件模型: /content/YOLOX/yolox_plate_op11_merged.onnx
大小: 9.10 MB

跳过（未找到）：/content/LPRNet_Pytorch/lprnet_standard_op11.onnx


In [ ]:
import torch
import onnx
import os
from yolox.exp import get_exp

# ================= 配置区 =================
# 请确保这些路径是对的
ckpt_file = "/content/YOLOX/latest_ckpt.pth"  # 你的权重文件
exp_file = "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py" # 你的配置文件
output_name = "yolox_plate_real_op11.onnx" #哪怕文件名叫这个
# =========================================

print("正在加载模型...")
# 1. 加载实验配置
exp = get_exp(exp_file, None)
model = exp.get_model()

# 2. 加载权重
ckpt = torch.load(ckpt_file, map_location="cpu")
model.load_state_dict(ckpt["model"])
model.eval()

# 3. 准备输入
dummy_input = torch.randn(1, 3, 640, 640)

# 4. 强制导出 Opset 11
print(f"正在强制导出为 Opset 11 到 {output_name} ...")
torch.onnx.export(
    model,
    dummy_input,
    output_name,
    input_names=["images"],
    output_names=["output"],
    dynamic_axes=None,
    opset_version=11,  # <--- 这里是绝对的命令，PyTorch 必须听
    keep_initializers_as_inputs=True # 优化选项
)

# 5. 立即验证
print("正在验证导出结果...")
onnx_model = onnx.load(output_name)
ver = onnx_model.opset_import[0].version
print(f"✅ 最终文件的内部版本是: Opset {ver}")

if ver == 11:
    print("🎉 成功！请下载这个 'yolox_plate_real_op11.onnx' 文件！")
else:
    print(f"😱 见鬼了，怎么还是 {ver}？请检查 PyTorch 版本。")

正在加载模型...
正在强制导出为 Opset 11 到 yolox_plate_real_op11.onnx ...


W1217 14:08:15.950000 293 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `YOLOX([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `YOLOX([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


[torch.onnx] Translate the graph into ONNX... ✅


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/BaseConverter.h:65: adapter_lookup: Assertion `false`

Applied 148 of general pattern rewrite rules.
正在验证导出结果...
✅ 最终文件的内部版本是: Opset 18
😱 见鬼了，怎么还是 18？请检查 PyTorch 版本。


In [5]:
import os
import sys

# ==========================================
# 1. 环境准备 (Miniforge)
# ==========================================
print("🧹 清理环境...")
%cd /content
!rm -rf /content/miniforge /content/py38_env
!rm -f miniforge.sh export_worker.py

print("📦 下载 Miniforge...")
!wget "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh" -O miniforge.sh

print("📦 安装 Miniforge...")
!bash miniforge.sh -b -p /content/miniforge

# ==========================================
# 2. 创建 Python 3.8 环境
# ==========================================
print("🐍 创建 Python 3.8 环境...")
!/content/miniforge/bin/conda create -p /content/py38_env python=3.8 pip -y

env_path = "/content/py38_env"
pip_path = os.path.join(env_path, "bin/pip")
python_path = os.path.join(env_path, "bin/python")

if not os.path.exists(pip_path):
    raise RuntimeError("❌ 环境创建失败，请重试。")

# ==========================================
# 3. 安装依赖 (全量版 - 拒绝打地鼠)
# ==========================================
print("⬇️ 1. 安装 PyTorch 1.12 (CPU版)...")
!{pip_path} install torch==1.12.1+cpu torchvision==0.13.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu

print("⬇️ 2. 安装 YOLOX 的所有潜在依赖 (全家桶)...")
# 这里把 packaging, matplotlib, pyyaml, scikit-image, tensorboard 等全部加上
# protobuf 指定 3.20.x 是为了防止 ONNX 兼容性问题
!{pip_path} install packaging matplotlib pyyaml scikit-image pycocotools requests scipy seaborn pandas tensorboard protobuf==3.20.3 future

print("⬇️ 3. 安装/锁定核心工具库...")
# 再次锁定 onnx 版本，防止被上面的库升级
!{pip_path} install onnx==1.12.0 loguru thop tabulate opencv-python psutil tqdm ninja

print("✅ 所有可能的依赖都装好了！")

# ==========================================
# 4. 执行导出
# ==========================================
yolox_dir = "/content/YOLOX"
ckpt_path = os.path.join(yolox_dir, "latest_ckpt.pth")
exp_path = os.path.join(yolox_dir, "exps/example/yolox_voc/yolox_voc_s.py")
output_onnx = "yolox_plate_py38_op11.onnx"

script_content = f"""
import torch
import onnx
import sys
import os

# 1. 添加 YOLOX 路径
sys.path.append("{yolox_dir}")

# 2. 尝试导入 (这里如果还缺包，会直接打印出来)
try:
    from yolox.exp import get_exp
    print("✅ YOLOX 库导入成功！")
except ImportError as e:
    print(f"❌ 依然缺包: {{e}}")
    sys.exit(1)

try:
    print(f"🔥 Python: {{sys.version.split()[0]}} | PyTorch: {{torch.__version__}}")

    # 加载
    exp = get_exp("{exp_path}", None)
    model = exp.get_model()
    ckpt = torch.load("{ckpt_path}", map_location="cpu")
    model.load_state_dict(ckpt["model"])
    model.eval()

    # 导出
    dummy_input = torch.randn(1, 3, 640, 640)
    print(f"🚀 开始导出 Opset 11 到 {{output_name}} ...")

    torch.onnx.export(
        model, dummy_input, "{output_onnx}",
        input_names=["images"], output_names=["output"],
        opset_version=11, do_constant_folding=True
    )

    # 验证
    m = onnx.load("{output_onnx}")
    ver = m.opset_import[0].version
    print(f"✅ 导出成功！最终内部版本: Opset {{ver}}")

except Exception as e:
    print(f"❌ 运行错误: {{e}}")
    import traceback
    traceback.print_exc()
"""

with open("export_worker.py", "w") as f:
    f.write(script_content)

print("🚀 运行导出脚本...")
!{python_path} export_worker.py

🧹 清理环境...
/content
📦 下载 Miniforge...
--2025-12-18 09:45:54--  https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/conda-forge/miniforge/releases/download/25.11.0-1/Miniforge3-Linux-x86_64.sh [following]
--2025-12-18 09:45:54--  https://github.com/conda-forge/miniforge/releases/download/25.11.0-1/Miniforge3-Linux-x86_64.sh
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/221584272/88c3b2ae-9e3f-448e-80be-e7a0cf00c0c1?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-18T10%3A29%3A49Z&rscd=attachment%3B+filename%3DMiniforge3-Linux-x86_64.sh&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6

In [ ]:
import os
import sys

# ==========================================
# 1. 依赖已就绪，直接跳到导出逻辑
# ==========================================
# 重新定义路径，防止变量丢失
yolox_dir = "/content/YOLOX"
ckpt_path = os.path.join(yolox_dir, "latest_ckpt.pth")
exp_path = os.path.join(yolox_dir, "exps/example/yolox_voc/yolox_voc_s.py")
output_onnx = "yolox_plate_py38_op11.onnx"

# 这一步是关键：指定我们刚才装好的环境路径
python_path = "/content/py38_env/bin/python"

# ==========================================
# 2. 修正后的导出脚本
# ==========================================
# 修复点：在脚本内部明确定义 output_name 变量

script_content = f"""
import torch
import onnx
import sys
import os

# 1. 添加 YOLOX 路径
sys.path.append("{yolox_dir}")

try:
    from yolox.exp import get_exp
    # 明确定义输出文件名变量，解决 NameError
    output_name = "{output_onnx}"

    print(f"🔥 Python: {{sys.version.split()[0]}}")

    # 加载
    exp = get_exp("{exp_path}", None)
    model = exp.get_model()
    ckpt = torch.load("{ckpt_path}", map_location="cpu")
    model.load_state_dict(ckpt["model"])
    model.eval()

    # 导出
    dummy_input = torch.randn(1, 3, 640, 640)
    print(f"🚀 开始导出 Opset 11 到 {{output_name}} ...")

    torch.onnx.export(
        model, dummy_input, output_name,
        input_names=["images"], output_names=["output"],
        opset_version=11, do_constant_folding=True
    )

    # 验证
    m = onnx.load(output_name)
    ver = m.opset_import[0].version
    print(f"✅ 导出成功！最终内部版本: Opset {{ver}}")
    print(f"🎉 文件已生成: {{output_name}}")

except Exception as e:
    print(f"❌ 运行错误: {{e}}")
    import traceback
    traceback.print_exc()
"""

with open("export_worker.py", "w") as f:
    f.write(script_content)

print("🚀 运行修复后的导出脚本...")
!{python_path} export_worker.py

🚀 运行修复后的导出脚本...
🔥 Python: 3.8.20
🚀 开始导出 Opset 11 到 yolox_plate_py38_op11.onnx ...
✅ 导出成功！最终内部版本: Opset 11
🎉 文件已生成: yolox_plate_py38_op11.onnx


In [ ]:
import os

# 1. 确保在 content 目录下
%cd /content

# 2. 克隆仓库
if os.path.exists("LPRNet_Pytorch"):
    !rm -rf LPRNet_Pytorch
!git clone https://github.com/sirius-ai/LPRNet_Pytorch.git

# 3. 下载权重
%cd /content/LPRNet_Pytorch
if not os.path.exists("lprnet_standard.pth"):
    print("⬇️ 下载预训练权重...")
    !wget https://github.com/sirius-ai/LPRNet_Pytorch/raw/master/weights/Final_LPRNet_model.pth -O lprnet_standard.pth

/content
Cloning into 'LPRNet_Pytorch'...
remote: Enumerating objects: 1071, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1071 (delta 25), reused 22 (delta 22), pack-reused 1037 (from 1)
Receiving objects: 100% (1071/1071), 20.04 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/LPRNet_Pytorch
⬇️ 下载预训练权重...
--2025-12-18 06:11:02--  https://github.com/sirius-ai/LPRNet_Pytorch/raw/master/weights/Final_LPRNet_model.pth
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sirius-ai/LPRNet_Pytorch/master/weights/Final_LPRNet_model.pth [following]
--2025-12-18 06:11:02--  https://raw.githubusercontent.com/sirius-ai/LPRNet_Pytorch/master/weights/Final_LPRNet_model.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.10

In [ ]:
import os
import sys

# ==========================================
# 1. 环境准备
# ==========================================
print("🧹 清理环境...")
%cd /content
!rm -rf LPRNet_Pytorch lprnet_*.onnx export_lpr_final_v5.py

print("📦 下载权重...")
!git clone https://github.com/sirius-ai/LPRNet_Pytorch.git
repo_dir = "/content/LPRNet_Pytorch"
ckpt_path = os.path.join(repo_dir, "lprnet_standard.pth")
if not os.path.exists(ckpt_path):
    !wget https://github.com/sirius-ai/LPRNet_Pytorch/raw/master/weights/Final_LPRNet_model.pth -O {ckpt_path}

# ==========================================
# 2. 编写“瘦身版”导出脚本
# ==========================================
output_onnx_raw = "lprnet_slim_raw.onnx"
output_onnx_sim = "lprnet_standard_op11_sim.onnx"
python_path = "/content/py38_env/bin/python"

script_content = f"""
import torch
import torch.nn as nn
import sys
import os
from collections import OrderedDict

# ==========================================================
# 1. 定义“瘦身版”模型 (Slim LPRNet)
#    通道数全程保持 64，适配官方权重
# ==========================================================
class SmallBasicBlock(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(SmallBasicBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)

class LPRNet_Slim(nn.Module):
    def __init__(self, class_num, dropout_rate):
        super(LPRNet_Slim, self).__init__()
        self.class_num = class_num
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1),

            # Block 1: 64 -> 64 (原代码是128)
            SmallBasicBlock(ch_in=64, ch_out=64),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),

            # MaxPool2d (H压缩)
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,1)),

            # Block 2: 64 -> 64 (原代码是256)
            SmallBasicBlock(ch_in=64, ch_out=64),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),

            # Block 3: 64 -> 64 (原代码是256)
            SmallBasicBlock(ch_in=64, ch_out=64),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),

            # MaxPool2d (H压缩)
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,1), padding=(0,1)),

            nn.Dropout(dropout_rate),

            # Layer 16: 输入64，输出256
            # 代码期望 (4,1)，权重是 (1,4) -> 待会旋转
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(4, 1), stride=1),

            nn.Dropout(dropout_rate),

            # Identity 顶替缺失的 BatchNorm
            nn.Identity(),

            nn.ReLU(),

            # Layer 20: 256 -> 68
            # 代码期望 (1,13), 权重是 (13,1) -> 待会旋转
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(1, 13), stride=1),

            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),
        )
        self.container = nn.Sequential(
            # 输入通道计算:
            # Global Context:
            # Layer 0(64) + Layer 1(64) + Layer 2(64) + Layer 3(64)
            # Total = 256 (不是448了)
            # 所以这里 container 输入必须是 256 + class_num
            nn.Conv2d(in_channels=256+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1))
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            # 索引需要微调，但在 Sequential 里遍历是一样的
            if i in [2, 6, 13, 22]:
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)
        return logits

# ==========================================================
# 2. 权重加载与旋转逻辑
# ==========================================================
try:
    print("🔥 初始化 Slim 版模型 (64通道)...")
    model = LPRNet_Slim(class_num=68, dropout_rate=0)
    current_state = model.state_dict()

    print(f"📦 加载权重...")
    loaded_state = torch.load("{ckpt_path}", map_location='cpu')
    clean_state = OrderedDict()
    for k, v in loaded_state.items():
        clean_state[k.replace('module.', '')] = v

    final_state = OrderedDict()

    for k, v_target in current_state.items():
        if "backbone.18" in k: continue # Skip Identity

        # 兼容 Container 权重 (因为通道数变了，Container 权重可能也需要修剪)
        if "container.0.weight" in k:
            v_source = clean_state[k]
            # 源码 container 输入是 448+68=516
            # 我们 container 输入是 256+68=324
            # 权重形状 [68, 516, 1, 1] -> [68, 324, 1, 1]
            if v_source.shape[1] == 516 and v_target.shape[1] == 324:
                print(f"✂️ 修剪 Container 权重: {{v_source.shape}} -> {{v_target.shape}}")
                # 假设前 64*4=256 是对应的，后面是 class_num
                # 这是一个大胆的假设，但对于迁移学习权重通常是这样
                # 我们取前 256 和 最后 68
                part1 = v_source[:, :256, :, :]
                part2 = v_source[:, 448:, :, :] # 最后的 class_num 部分
                final_state[k] = torch.cat([part1, part2], dim=1)
                continue

        if k not in clean_state:
            print(f"⚠️ 警告: 缺失 {{k}}")
            continue

        v_source = clean_state[k]

        # 修复 Layer 16: (1, 4) -> (4, 1)
        if "backbone.16.weight" in k:
            print(f"🔄 旋转 Layer 16: {{v_source.shape}} -> {{v_target.shape}}")
            final_state[k] = v_source.permute(0, 1, 3, 2)

        # 修复 Layer 20: (13, 1) -> (1, 13)
        elif "backbone.20.weight" in k:
            print(f"🔄 旋转 Layer 20: {{v_source.shape}} -> {{v_target.shape}}")
            final_state[k] = v_source.permute(0, 1, 3, 2)

        else:
            final_state[k] = v_source

    model.load_state_dict(final_state, strict=False)
    print("✅ 权重加载成功！")
    model.eval()

    # 导出
    dummy_input = torch.randn(1, 3, 24, 94)
    output_name = "{output_onnx_raw}"

    print(f"🚀 开始导出...")
    torch.onnx.export(
        model, dummy_input, output_name,
        input_names=["input"], output_names=["output"],
        opset_version=11,
        do_constant_folding=True
    )
    print("✅ 导出成功")

except Exception as e:
    print(f"❌ 失败: {{e}}")
    import traceback
    traceback.print_exc()
"""

with open("export_lpr_final_v5.py", "w") as f:
    f.write(script_content)

print("\n🚀 运行 V5 脚本...")
!{python_path} export_lpr_final_v5.py

# 简化
if os.path.exists(output_onnx_raw):
    print(f"\n✂️ 正在简化...")
    !{python_path} -m onnxsim {output_onnx_raw} {output_onnx_sim} --overwrite-input-shape "1,3,24,94"
    print(f"\n🎉 真的真的真的完成了！下载: {output_onnx_sim}")
else:
    print("❌ 失败")

🧹 清理环境...
/content
📦 下载权重...
Cloning into 'LPRNet_Pytorch'...
remote: Enumerating objects: 1071, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1071 (delta 25), reused 22 (delta 22), pack-reused 1037 (from 1)
Receiving objects: 100% (1071/1071), 20.04 MiB | 26.75 MiB/s, done.
Resolving deltas: 100% (35/35), done.
--2025-12-18 06:16:09--  https://github.com/sirius-ai/LPRNet_Pytorch/raw/master/weights/Final_LPRNet_model.pth
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sirius-ai/LPRNet_Pytorch/master/weights/Final_LPRNet_model.pth [following]
--2025-12-18 06:16:09--  https://raw.githubusercontent.com/sirius-ai/LPRNet_Pytorch/master/weights/Final_LPRNet_model.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109

In [ ]:
import onnx
model = onnx.load("lprnet_standard_op11_sim.onnx")
# 打印所有节点的类型，确保只有 MaxPool (2D语义)，没有 MaxPool3d
ops = set([node.op_type for node in model.graph.node])
print("模型中包含的算子:", ops)
if "MaxPool" in ops and "MaxPool3d" not in ops:
    print("✅ 检测通过：只有标准 MaxPool，没有 3D 版本！")

In [6]:
import os
import sys

# ==========================================
# 1. 环境准备 (Py3.8 + PyTorch 1.12)
# ==========================================
python_path = "/content/py38_env/bin/python"

# 你的 YOLOX 权重路径 (请务必确认文件存在！)
# 建议直接用左侧文件栏右键 'Copy Path' 复制过来
ckpt_path = "/content/YOLOX/latest_ckpt.pth"
yolox_dir = "/content/YOLOX"
output_onnx = "yolox_plate_py38_op11_fixed.onnx"

# ==========================================
# 2. 编写带校验的导出脚本
# ==========================================
script_content = f"""
import torch
import onnx
import sys
import os

sys.path.append("{yolox_dir}")
from yolox.exp import get_exp

try:
    print(f"🔥 正在加载实验配置...")
    exp = get_exp("{os.path.join(yolox_dir, 'exps/example/yolox_voc/yolox_voc_s.py')}", None)
    model = exp.get_model()

    print(f"📦 正在加载权重: {ckpt_path}")
    if not os.path.exists("{ckpt_path}"):
        raise FileNotFoundError("❌ 找不到权重文件！请检查路径！")

    ckpt = torch.load("{ckpt_path}", map_location="cpu")

    # 【关键校验】确保权重真的加载进去了
    if "model" in ckpt:
        load_res = model.load_state_dict(ckpt["model"], strict=True)
    else:
        load_res = model.load_state_dict(ckpt, strict=True)

    print(f"✅ 权重加载结果: {{load_res}}")
    model.eval()

    # 导出
    dummy_input = torch.randn(1, 3, 640, 640)
    output_name = "{output_onnx}"

    print(f"🚀 开始导出 Opset 11...")
    torch.onnx.export(
        model,
        dummy_input,
        output_name,
        input_names=["images"],
        output_names=["output"],
        opset_version=11,
        do_constant_folding=True
    )

    # 验证 Opset
    m = onnx.load(output_name)
    ver = m.opset_import[0].version
    print(f"✅ 导出成功！最终内部版本: Opset {{ver}}")

except Exception as e:
    print(f"❌ 严重错误: {{e}}")
    sys.exit(1)
"""

with open("export_yolox_verify.py", "w") as f:
    f.write(script_content)

print("🚀 运行校验版导出脚本...")
!{python_path} export_yolox_verify.py

🚀 运行校验版导出脚本...
🔥 正在加载实验配置...
📦 正在加载权重: /content/YOLOX/latest_ckpt.pth
✅ 权重加载结果: <All keys matched successfully>
🚀 开始导出 Opset 11...
✅ 导出成功！最终内部版本: Opset 11
